In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso, Ridge, LassoCV, LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("energydata_complete.csv")
df.head(2)

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.7900,19.79,44.73,19.0,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.7225,19.79,44.79,19.0,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195


### Instruction

To answer some questions, you will need to normalize the dataset using the MinMaxScaler after removing the following columns: [“date”, “lights”].

The target variable is “Appliances”. Use a 70-30 train-test set split with a random state of 42 (for reproducibility). Run a multiple linear regression using the training set and evaluate your model on the test set.

In [3]:
df = df.drop(columns = ["date", "lights"])
df.columns

Index(['Appliances', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4',
       'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9',
       'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility',
       'Tdewpoint', 'rv1', 'rv2'],
      dtype='object')

In [4]:
target = df["Appliances"].values
feat = df.drop(columns = ["Appliances"]).values

In [5]:
scalar = MinMaxScaler()

feat = scalar.fit_transform(feat)

In [6]:
trainx, testx, trainy, testy = train_test_split(feat, target, train_size= 0.7, random_state=42)
print("Train, Test sizes:", len( trainx ), len( trainy ))

Train, Test sizes: 13814 13814


### Question 12
From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the R^2 value in two D.P?

In [7]:
from scipy import stats
import seaborn as sns
x  = "T2"
y = "T6"

func = lambda x,y : stats.pearsonr(x, y)[0] ** 2

print(func(df[x], df[y]))

0.6418990830855483


In [8]:
inx, iny, outx, outy = train_test_split(df[x], df[y],train_size=0.7, random_state=42)

greg = LinearRegression()
greg.fit(inx.values.reshape(1, -1), iny.values.reshape(1, -1))

LinearRegression()

### Fitting model for the last few questions

Assuming the last questions are based off of the energy data

In [9]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
last_func = lambda x, y: np.sum(np.square(x - y))

In [10]:
reg = LinearRegression()
reg.fit(trainx, trainy)

LinearRegression()

In [11]:
val0 = testy
val1 = reg.predict(testx)

### Question 13
What is the Mean Absolute Error (in two decimal places)?

In [12]:
mean_absolute_error(val0, val1)

53.6411221799174

### Question 14

What is the Residual Sum of Squares (in two decimal places)?

In [13]:
last_func(val0, val1)

51917116.49637169

### Question 15

What is the Root Mean Squared Error (in three decimal places)?

In [14]:
mean_squared_error(val0, val1)

8768.302059849972

### Question 16

What is the Coefficient of Determination (in two decimal places)?

In [15]:
r2_score(val0, val1)

0.1489251627747067

In [16]:
np.argmax(reg.coef_)

25

### Question 17


Obtain the feature weights from your linear model above. Which features have the lowest and highest weights respectively?

In [17]:
df.columns[25]

'rv1'

### Question 18

Train a ridge regression model with an alpha value of 0.4. Is there any change to the root mean squared error (RMSE) when evaluated on the test set?

In [18]:
ridge = Ridge(alpha = 0.4)
ridge.fit(trainx, trainy, )
mean_squared_error(testy, ridge.predict(testx))

8772.425450548444

### Question 19 & 20

Train a lasso regression model with an alpha value of 0.001 and obtain the new feature weights with it. How many of the features have non-zero feature weights?

In [19]:
reg = Lasso(alpha = 0.001)
reg.fit(trainx, trainy, )
reg.coef_
print( mean_squared_error(testy, reg.predict(testx)) )

8768.576031658373
